After reading chapter 2 of book I reached the Try it out section of the book. I am to pick any dataset I want and use it to practice the data science cycle taught by this chapter.
I chose kaggle titanic dataset. The goal is to "use machine learning to create a model that predicts which passengers survived the Titanic shipwreck."

In [2]:
import kaggle
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import numpy as np

In [3]:
def load_titanic_train_test_set():
    if not Path("datasets/titanic").is_dir():
        Path("datasets").mkdir(exist_ok=True)
        kaggle.api.authenticate()
        kaggle.api.competition_download_files("titanic", path="./datasets", force=False, quiet=True)
    
        with ZipFile("datasets/titanic.zip", 'r') as titanic_zip:
            titanic_zip.extractall("datasets/titanic")
    
    return pd.read_csv("datasets/titanic/train.csv", index_col="PassengerId"), pd.read_csv("datasets/titanic/test.csv",index_col="PassengerId")


In [4]:
train, test = load_titanic_train_test_set()

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (891, 11)
Test shape: (418, 10)


## Explore Data

In [5]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


    Convert sex to a binary type column
    Age has missing values
    cabin has a lot of missing values, see correlation with survived
    find out what is relevant in name and ticket (if they are relevant)
    

In [7]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [26]:
# inspect non-null cabin values
train[train["Pclass"] == 1][train["Survived"] == 1]["Cabin"].value_counts() # when Pclass == 1

C:\Users\ianvi\AppData\Local\Temp\ipykernel_18272\504330178.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train[train["Pclass"] == 1][train["Survived"] == 1]["Cabin"].value_counts() # when Pclass == 1


Cabin
B96 B98            4
C92                2
D36                2
B57 B59 B63 B66    2
B5                 2
                  ..
E40                1
C32                1
E34                1
C7                 1
C148               1
Name: count, Length: 91, dtype: int64

In [27]:
train[train["Pclass"] == 1][train["Survived"] == 0]["Cabin"].value_counts() # when Pclass == 1

C:\Users\ianvi\AppData\Local\Temp\ipykernel_18272\844495181.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train[train["Pclass"] == 1][train["Survived"] == 0]["Cabin"].value_counts() # when Pclass == 1


Cabin
C23 C25 C27    2
D26            2
C124           2
C22 C26        2
E46            1
D50            1
A14            1
B37            1
C30            1
C65            1
D46            1
C95            1
B38            1
C86            1
A10            1
D48            1
E38            1
E58            1
B71            1
C68            1
C46            1
B22            1
D30            1
D6             1
B82 B84        1
A36            1
B102           1
A24            1
E63            1
C82            1
C111           1
C49            1
B30            1
C83            1
E31            1
A5             1
C110           1
B58 B60        1
C123           1
B86            1
B19            1
A7             1
A32            1
E44            1
C78            1
C87            1
E67            1
B94            1
C118           1
A19            1
C91            1
C2             1
T              1
C128           1
B51 B53 B55    1
Name: count, dtype: int64

In [22]:
train[train["Pclass"] == 2][["Survived","Cabin"]].value_counts() # when Pclass == 2

Survived  Cabin
1         E101     3
          F33      3
          D        2
          F2       2
          F4       2
0         D        1
          E77      1
          F2       1
1         D56      1
Name: count, dtype: int64

In [21]:
train[train["Pclass"] == 3][["Survived","Cabin"]].value_counts() # when Pclass == 3

Survived  Cabin
0         F G73    2
          G6       2
1         E121     2
          G6       2
0         F G63    1
          F38      1
1         E10      1
          F E69    1
Name: count, dtype: int64